In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T064849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D68f9471a1787b8360c85c7eb687ef0920cbc628b30905cc04d593e7c01a70c55452042dfaf444c18cf6f0486727740ce32f8f45f59c7221113bdbc5492ed418d17e7e9a190015a12b0ac01fd0f48df21ca0acbe5a0763d5b836b2e7275c2b243f092bc391beea7116ad3950f57bb927b7a874d3f55915aeb1120686507a26c723901cd3226ac4298299501ed26bebde89601edaa749ed5a5e3c9f7782c4a4a595173ec62d8d1ec22164a679632c5310f15903db4d58fb40805794efe0ca1fd7ffc04c06e3ef523b600cd1d99d469fe0086f815753a91add0b71f04a9fc0766a841d1724cf3b7d40f34770f2b005c1f12ff617df6e6aed72b7bd3ab0a688137ea,data-science-qna-llm-fine-tuning:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4500171%2F7707625%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T064849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6d8236cf7b4d02603d717440ea3f841cca75f34b2d550eea348aa487d1bb523220b714644c2ffdb5c00b1ac213b2b215633080b91601dc1760478b3834285aeaaca81698ea62763261a40ff0a952bfc71a263c3ec313b4d7c16b4bf9063d05b841157bf0f06068a8d96ec5200ef2cb5201050c3d91c5e8fc972684e780302164cc7516ea751845ece381acd27dfbf1c5ddd24e056f1dd2866de261b1872bce639e92539d2bca6136af2b6e8b5cd817b655b69eb0f177bb7f2f47736cbbc3a5bee84dd3a51714765cebc1d5ed5c99a1da30ece73409efc1bec8261357411507257d45c07440f26c21b5b65e7d227505888bdd1c623d4186b300e108e12f295554,gemma/keras/gemma_2b_en/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F5171%2F11371%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240304%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240304T064849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D13d638ac47b080198b0ed35346b46f77b97bd369a9982a9421d94d3e45ee6d1b017696811e760e1b1496aa38f3fe8fa197c37a815d34f04ef65941dcd429cf4501d8148aec014b75000b4d38afba645ece075b1d305b7e9040cb355c07f7038d0d3019c5e15d31b77af65ebd8f7b18a60fd7f0e32554f7ef66067f9d770d7ee1c03deda249173a56f5f24ffc3ab2e49039d0c5881bccd18141b16f5984fef039328d2fcee9f3f1bba93965b72dd82337aabe7295959a25be2839ac74d2eca729562e3bd528354cc8426fe58a95ac34ae60ec217ce28c9e044042862b0a7d01d58df44d32640e078a96ba828b61314915905765500ea68b63ff3c827c578fa9e5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 1183 bytes downloaded
Downloaded and uncompressed: data-assistants-with-gemma
[==================================================] 83058 bytes downloaded
Downloaded and uncompressed: data-science-qna-llm-fine-tuning
[==================================================] 4014312183 bytes downloaded
Downloaded and uncompressed: gemma/keras/gemma_2b_en/2
Data source import complete.


In [ ]:
!pip install accelerate peft bitsandbytes trl
!pip -q install git+https://github.com/huggingface/transformers.git

# Upgrade to python 3.11
!mamba create -n py311 -y
!source /opt/conda/bin/activate py311 && mamba install python=3.11 jupyter mamba -y

!sudo rm /opt/conda/bin/python3
!sudo ln -sf /opt/conda/envs/py311/bin/python3 /opt/conda/bin/python3
!sudo rm /opt/conda/bin/python3.7
!sudo ln -sf /opt/conda/envs/py311/bin/python3 /opt/conda/bin/python3.7
!sudo rm /opt/conda/bin/python
!sudo ln -sf /opt/conda/envs/py311/bin/python3 /opt/conda/bin/python



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
/bin/bash: line 1: mamba: command not found
/bin/bash: line 1: /opt/conda/bin/activate: No such file or directory
rm: cannot remove '/opt/conda/bin/python3': No such file or directory
ln: failed to create 

In [ ]:
!pip install -U "huggingface_hub[cli]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install -U datasets

In [ ]:
import tensorflow as tf
print(tf.__version__)
#!pip install tf-nightly

2.15.0


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/data-science-qna-llm-fine-tuning/datascienceQnA.csv
/kaggle/input/gemma/keras/gemma_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/data-assistants-with-gemma/submission_instructions.txt
/kaggle/input/data-assistants-with-gemma/submission_categories.txt


In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [ ]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = 'yoonjiyoung'
os.environ['KAGGLE_KEY'] = ''

In [ ]:
import pandas as pd
import os
import torch

# Tools for Fine Tuning provided by hugging face transformers


# Fine Tuning Frameworks
from peft import LoraConfig
from trl import SFTTrainer

# sklearn and Datasets for processing the train & test data
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [ ]:
!pip install kaggle_secrets

ERROR: Could not find a version that satisfies the requirement kaggle_secrets (from versions: none)
ERROR: No matching distribution found for kaggle_secrets


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-probability


In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import BitsAndBytesConfig
from transformers import TrainingArguments
from transformers import pipeline
from transformers import logging

In [ ]:
# This function helps format the dataset, so that it can be used for fine-tuning.
# We will use alpaca style formatting for this project
def create_instruction(row):
    instruction_template = """Explain detail this python code.
### Instruction: {}
### Response: {}""".format(row['output'], row['input'])
    return instruction_template


In [ ]:
# Import Dataset
#df = pd.read_csv("/kaggle/input/data-science-qna-llm-fine-tuning/datascienceQnA.csv")
#df

# Double-check if there is ~25k examples instead of almost 50k (HF thinks jsonl and json are two different datasets)

from datasets import load_dataset

dataset = load_dataset('flytech/python-codes-25k',split='train')

# One can map the dataset in any way, for the sake of example:
df = pd.DataFrame(dataset)
df.head()

,output,input,text,instruction
0,```python\ntasks = []\nwhile True:\n task =...,Setting up your daily to-do list...,Help me set up my daily to-do list! Setting up...,Help me set up my daily to-do list!
1,```python\nshopping_list = {}\nwhile True:\n ...,Creating a shopping list...,Create a shopping list based on my inputs! Cre...,Create a shopping list based on my inputs!
2,"```python\ntotal_time = 0\nfor i in range(1, 8...",Calculating weekly phone usage...,Calculate how much time I spend on my phone pe...,Calculate how much time I spend on my phone pe...
3,```python\ntotal_bill = float(input('Enter the...,Splitting the bill...,Help me split the bill among my friends! Split...,Help me split the bill among my friends!
4,```python\nmovie_list = {}\nwhile True:\n g...,Organizing your movie list...,Organize my movie list into genres! Organizing...,Organize my movie list into genres!


In [ ]:
# To convert to alpaca we need to add text column
df['text'] = df.apply(create_instruction, axis=1)
df

,output,input,text,instruction
0,```python\ntasks = []\nwhile True:\n task =...,Setting up your daily to-do list...,Explain detail this python code.\n### Instruct...,Help me set up my daily to-do list!
1,```python\nshopping_list = {}\nwhile True:\n ...,Creating a shopping list...,Explain detail this python code.\n### Instruct...,Create a shopping list based on my inputs!
2,"```python\ntotal_time = 0\nfor i in range(1, 8...",Calculating weekly phone usage...,Explain detail this python code.\n### Instruct...,Calculate how much time I spend on my phone pe...
3,```python\ntotal_bill = float(input('Enter the...,Splitting the bill...,Explain detail this python code.\n### Instruct...,Help me split the bill among my friends!
4,```python\nmovie_list = {}\nwhile True:\n g...,Organizing your movie list...,Explain detail this python code.\n### Instruct...,Organize my movie list into genres!
...,...,...,...,...
49621,c.execute('''CREATE TABLE IF NOT EXISTS stocks...,Creating a 'stocks' table...,Explain detail this python code.\n### Instruct...,Execute code: c.execute('''CREATE TABLE IF NOT...
49622,"c.execute(""INSERT INTO stocks VALUES ('2023-09...",Inserting a record into the 'stocks' table...,Explain detail this python code.\n### Instruct...,"Execute code: c.execute(""INSERT INTO stocks VA..."
49623,c.execute('SELECT * FROM stocks WHERE symbol=?...,Querying the 'stocks' table for records with s...,Explain detail this python code.\n### Instruct...,Execute code: c.execute('SELECT * FROM stocks ...
49624,c.execute('UPDATE stocks SET qty=120 WHERE sym...,Updating the 'stocks' table to change quantity...,Explain detail this python code.\n### Instruct...,Execute code: c.execute('UPDATE stocks SET qty...


In [ ]:
# Example
df['text'].iloc[[3]].values[0]

"Explain detail this python code.\n### Instruction: ```python\ntotal_bill = float(input('Enter the total bill amount: '))\nfriends_count = int(input('Enter the number of friends: '))\nper_person = total_bill / friends_count\nprint(f'Each person should pay {per_person}')\n```\n### Response: Splitting the bill..."

In [ ]:
# Convert dataset to hugging face format
dataset = Dataset.from_pandas(df[['output','input','text']])

In [ ]:
# Model Defining
base_model = "google/gemma-2b"
mew_model = "google/gemma-2b-python_code_100"

In [ ]:
pip install bitsandbytes

In [ ]:
#import transformers
# Using Lora to tune only limited parameters keeping resources in mind
lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# version 문제로 인해서 install 해주어야 함
!pip install 'git+https://github.com/keras-team/keras-nlp'

In [ ]:
import keras
import keras_nlp
import numpy as np

#keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
#  kerass api로 한 경우

#gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
#gemma_lm.generate("Keras is a", max_length=30)

 #Generate with batched prompts.
#gemma_lm.generate(["Keras is a", "I want to say"], max_length=30)

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
I0000 00:00:1709518143.580148     521 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


['Keras is a popular deep learning library for Python. It is easy to use and has a wide range of features. One of the features of',
 'I want to say thank you to the staff at the <strong><em><u><strong><em><u><strong><em><u><strong><em><u><strong><em><u><strong><em>']

일지_팀명_회차
매주 금요일 프로젝트 평가지 , 연구 노트 제출해도 가능(대체가능)
4월 12일 16시
포스터 권장, 최종 제출할 때
발표 전 주에 리허설을 한다.
4월 15일 월요일 - 최종 발표

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

In [ ]:
#from transformers import AutoTokenizer, AutoModelForCausalLM

# Define Tokenizer and Model from Hugging Face
#
model = AutoModelForCausalLM.from_pretrained(
                        base_model,
                        quantization_config=bnb_config,
                        device_map={"":0}
)


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Sample Conversation 1
# Complexity : Mid Level
text = "python\ntasks = []\nwhile True:\n    task = input('Enter a task or type 'done' to finish: ')\n    if task == 'done': break\n    tasks.append(task)\nprint(f'Your to-do list for today: {tasks}')\n"

device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

python
tasks = []
while True:
    task = input('Enter a task or type 'done' to finish: ')
    if task == 'done': break
    tasks.append(task)
print(f'Your to-do list for today: {tasks}')



In [ ]:
# Sample Conversation 2
# Complexity : Low Level
text = "with torch . no_grad (): a -= learning_rate * a . grad b -= learning_rate * b . grad c -= learning_rate * c . grad d -= learning_rate * d ."
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

with torch . no_grad (): a -= learning_rate * a . grad b -= learning_rate * b . grad c -= learning_rate * c . grad d -= learning_rate * d . grad

<code>def __init__(self, a, b, c, d):
    self.a = a
    self.b = b
    self.c = c
    self.d = d

    self.a_grad = torch . Tensor (a . grad)
    self.b_grad = torch . Tensor (b . grad)
    self.c_grad = torch . Tensor (c . grad)
    self.d_grad = torch . Tensor (d . grad)

    self.a_grad = torch . Tensor (a . grad)
    self.b_grad = torch . Tensor (b . grad)
    self.c_grad = torch . Tensor (c . grad)
    self.d_grad = torch . Tensor (d . grad)

    self.a_grad = torch . Tensor (a . grad)
    self.b_grad = torch . Tensor (b . grad)
    self.c_grad = torch . Tensor (c . grad)
    self.d_grad = torch . Tensor (d . grad)

    self.a_grad = torch . Tensor (a . grad)
    self


In [ ]:
# Sample Conversation 3
# Comlexity : Mid-high Level
text = "model = torch . nn . Sequential ( torch . nn . Linear ( 3 , 1 ), torch . nn . Flatten ( 0 , 1 ) ) loss_fn = torch . nn . MSELoss ( reduction = 'sum' ) what is Topic of this code?"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

model = torch . nn . Sequential ( torch . nn . Linear ( 3 , 1 ), torch . nn . Flatten ( 0 , 1 ) ) loss_fn = torch . nn . MSELoss ( reduction = 'sum' ) what is Topic of this code?

Answer:

The code is a model that takes in a
3-dimensional input and outputs a 1-dimensional output.


In [ ]:
# Defining all Training hyperparameters, Epoch = 2, optimizer = 8 bit to save memory
# If given more time, can be trained on 5-6 epochs for better results
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    optim="adamw_bnb_8bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
# Trainer Configuration
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:161: UserWarning: You passed a model_id to the SFTTrainer. This will automatically create an `AutoModelForCausalLM` or a `PeftModel` (if you passed a `peft_config`) for you.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/49626 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:294: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
# Used to clear cache memory for a fresh start
# enable lora backbone load_weight,
torch.cuda.empty_cache()

In [ ]:
# Train
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,1.034000
50,0.826800
75,0.831900
100,1.016700
125,0.639500
150,0.854000
175,0.710300
200,0.807200
225,0.787400
250,0.758400


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

KeyboardInterrupt: 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Saving the model
trainer.model.save_pretrained(mew_model)
trainer.tokenizer.save_pretrained(mew_model)

('google/gemma-2b-python_code_100/tokenizer_config.json',
 'google/gemma-2b-python_code_100/special_tokens_map.json',
 'google/gemma-2b-python_code_100/tokenizer.model',
 'google/gemma-2b-python_code_100/added_tokens.json',
 'google/gemma-2b-python_code_100/tokenizer.json')

<!-- # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session -->

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(mew_model)
model = AutoModelForCausalLM.from_pretrained(
                        mew_model,
                        quantization_config=bnb_config,
                        device_map={"":0},
                        )


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Sample Conversation 1 After Training
# Complexity : Mid Level
text = """loss = ( y_pred - y ) . pow ( 2 ) . sum () if t % 100 == 99 : print ( t , loss . item ()) what is Topic of this code?"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

loss = ( y_pred - y ) . pow ( 2 ) . sum () if t % 100 == 99 : print ( t , loss . item ()) what is Topic of this code?
### Instruction: ```python
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import Callback
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from s

In [ ]:
# Sample Conversation 2
# Complexity : Low Level
text = """shopping_list = {}\nwhile True:\n    item = input('Enter an item or type 'done' to finish: ')\n    if item == 'done': break\n    quantity = input(f'Enter the quantity for {item}: ')\n    shopping_list[item] = quantity\nprint(f'Your shopping list: {shopping_list}')\n"""
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

shopping_list = {}
while True:
    item = input('Enter an item or type 'done' to finish: ')
    if item == 'done': break
    quantity = input(f'Enter the quantity for {item}: ')
    shopping_list[item] = quantity
print(f'Your shopping list: {shopping_list}')



In [ ]:
# Sample Conversation 2
# Complexity : Low Level
text = "with torch . no_grad (): a -= learning_rate * a . grad b -= learning_rate * b . grad c -= learning_rate * c . grad d -= learning_rate * d ."
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

with torch . no_grad (): a -= learning_rate * a . grad b -= learning_rate * b . grad c -= learning_rate * c . grad d -= learning_rate * d . grad

def update_parameters ( a, b, c, d, learning_rate ):

with torch . no_grad (): a -= learning_rate * a . grad b -= learning_rate * b . grad c -= learning_rate * c . grad d -= learning_rate * d . grad

def update_parameters ( a, b, c, d, learning_rate ):

with torch . no_grad (): a -= learning_rate * a . grad b -= learning_rate * b . grad c -= learning_rate * c . grad d -= learning_rate * d . grad

def update_parameters ( a, b, c, d, learning_rate ):

with torch . no_grad (): a -= learning_rate * a . grad b -= learning_rate * b . grad c -= learning_rate * c . grad d -= learning_rate * d . grad

def update_parameters ( a, b, c, d, learning_rate ):

with torch . no_grad (): a -= learning_rate * a . grad b -= learning_rate * b . grad c -= learning_rate * c . grad d -= learning_rate * d . grad

def update_parameters ( a


In [ ]:
# Sample Conversation 3
# Comlexity : Mid-high Level
text = "model = torch . nn . Sequential ( torch . nn . Linear ( 3 , 1 ), torch . nn . Flatten ( 0 , 1 ) ) loss_fn = torch . nn . MSELoss ( reduction = 'sum' ) what is Topic of this code?"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=256)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

model = torch . nn . Sequential ( torch . nn . Linear ( 3 , 1 ), torch . nn . Flatten ( 0 , 1 ) ) loss_fn = torch . nn . MSELoss ( reduction = 'sum' ) what is Topic of this code?
### Instruction: ```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = nn.Linear(3, 1)
        self.flatten = nn.Flatten(0, 1)

    def forward(self, x):
        x = self.linear(x)
        x = self.flatten(x)
        return x
```
### Response: 
The code is a simple neural network model for classification. The model takes an input of 3 features and outputs a single class label. The model uses a linear layer to map the input features to a single output class label. The model is then flattened to reduce the dimensionality of the input features. The model is then used to classify the input data.
### Topic: 
Neural Networks for Classification
### Response: 
Neural networks are a type o

# After Fine tuning, we are getting relevent information but in a good format.
# To solve this we must train it further in a better way with summarization dataset.